In [6]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..\\..")
sys.path.append(PROJECT_ROOT)

In [7]:
from src.engines.bigquery_engine import run_query, run_update_query, write_df_to_bq

In [8]:
query="SELECT Company, SUM(Laid_Off_Count) AS tot_layoffs FROM `data-analysis-project-481112.dataset_collection.layoffs` WHERE Laid_Off_Count IS NOT NULL AND EXTRACT(YEAR FROM `Date`)=2024 GROUP BY Company ORDER BY tot_layoffs DESC LIMIT 20"

In [9]:
df_test=run_query(query)

d:\Data Analysis\Capstone project - Personal\projectvenv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [10]:
df_test.head(20)

,Company,tot_layoffs
0,Tesla,14000
1,SAP,8000
2,Dell,6000
3,Cisco,4250
4,Xerox,3000
5,PayPal,2500
6,Farfetch,2000
7,Unity,1800
8,Wayfair,1650
9,Expedia,1500


In [11]:
comp_list=list(df_test['Company'])


In [12]:
query2="SELECT company_clean, job_title FROM `data-analysis-project-481112.dataset_collection.luke_tech_jobs`"

In [13]:
df_jobs=run_query(query2)

d:\Data Analysis\Capstone project - Personal\projectvenv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [14]:
condition = df_jobs['company_clean'].str.contains('|'.join(comp_list), case=False, na=False)

In [15]:
df_jobs_filter=df_jobs[condition]

In [16]:
def normalize(x):
    if not isinstance(x, str):
        return x
    for k in comp_list:
        if k.lower() in x.lower():
            return k
    return x

df_jobs_filter['company_std'] = df_jobs_filter['company_clean'].apply(normalize)


C:\Users\Asus\AppData\Local\Temp\ipykernel_19256\3612415065.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jobs_filter['company_std'] = df_jobs_filter['company_clean'].apply(normalize)


In [17]:
df_jobs_final=df_jobs_filter[['company_std','job_title']].groupby(['company_std'], observed=True).agg(job_post_count=('job_title','count')).reset_index()

In [18]:
df_joined = df_test.merge(
    df_jobs_final,
    left_on='Company',
    right_on='company_std',
    how='left'
)
df_joined = df_joined.drop(columns=['company_std'])

In [19]:
q1=open('../sql/dashboard_5/d5_layoff_v_time.sql').read()
q2=open('../sql/dashboard_5/d5_layoff_v_unemployment.sql').read()


In [20]:
df_layoff_timeline=run_query(q1)
df_layoff_v_unemployment=run_query(q2)

d:\Data Analysis\Capstone project - Personal\projectvenv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [21]:
write_df_to_bq(df_layoff_timeline, dataset="dataset_collection", table="d5_layoff_timeline")
write_df_to_bq(df_layoff_v_unemployment, dataset="dataset_collection", table="d5_layoff_v_unemployment")
write_df_to_bq(df_joined, dataset="dataset_collection", table="d5_layoff_v_jobpostings")

d:\Data Analysis\Capstone project - Personal\projectvenv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(
d:\Data Analysis\Capstone project - Personal\projectvenv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(
d:\Data Analysis\Capstone project - Personal\projectvenv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


In [22]:
q3=open('../sql/dashboard_5/d5_update_query.sql').read()

In [23]:
run_update_query(q3)
print("Table updated !")

Table updated !
